In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import fsolve
import plotly.express as px

import sys
sys.path.append('../')
import plotting

In [ ]:
params_primer = {'H': -158, 'S': -445}
params_motif = {'H': -60.4, 'S': -185}

def dG(params, T):
    return params['H'] - ((273.15 + T) * params['S'])/1000

def K(params, T):
    return np.exp(-4184*dG(params, T)/(8.314*(273.15 + T)))

In [ ]:
def model(result, params_primer, params_motif, T, c_DNA_tot, c_primer):
    K_primer = K(params_primer, T)
    K_motif = K(params_motif, T)
    c_DNA = result[0] * c_DNA_tot
    c_DNA_blocked = result[1] * c_DNA_tot
    c_DNA_primer = result[2] * c_DNA_tot
    return [
        c_DNA_tot - c_DNA - c_DNA_blocked - c_DNA_primer,
        c_DNA - c_DNA_blocked/K_motif,
        c_DNA - c_DNA_primer/(K_primer*c_primer)
    ]

def solve(T, c_DNA_tot, c_primer):
    return fsolve(model, [0.5, 0.1, 0.4], args=(params_primer, params_motif, T, c_DNA_tot, c_primer))

T = np.linspace(45, 80, -15-50)
c_DNA_tot = 1e-10 # 100 pM
c_primer = 10e-6*1/20 # 10 uM * 1 uL / 20 uL

res = np.array([solve(t, c_DNA_tot, c_primer) for t in T]).T
df = pd.DataFrame({
    'T': T,
    'Free DNA': res[0],
    'Self-annealed DNA': res[1],
    'DNA-primer complex': res[2]
})
df

In [ ]:
plot_df = df.melt(id_vars='T', var_name='species', value_name='ratio')

fig = px.line(
    plot_df,
    x='T', 
    y='ratio',
    color='species',
    labels={'ratio': 'Fraction of DNA', 'T': 'Temperature (°C)', 'species': 'Species'},
)

fig.update_layout(
    width=320,
    height=200,
    margin=dict(l=0, r=10, t=10, b=0),
    showlegend=False,
)
fig.update_yaxes(range=[0, 1.01])

fig = plotting.standardize_plot(fig)
fig.show()
fig.write_image('SI_figure_correlation/distribution.svg')